# Grid Search Practice

Grid search is a way of finding better hypermeters, those that define the configuration of the model and not altered by learning of the model.
It is also known as Exhaustive Grid Search, because it takes many parameter options and creates a search space with cartesian product and then exhaust and evaluate all of these possibilities.
In essence, it is a brute force algorithm.
Let's get familiarize with its usage through practice.

First, we prepare some data.

In [2]:
from time import time
import os, itertools
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

DATASET = '/dsa/data/all_datasets/titanic_ML/titanic.csv'
assert os.path.exists(DATASET)

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

dataset = pd.read_csv(DATASET).sample(frac = 1).reset_index(drop=True)
X = dataset.iloc[:, :-1]
y = dataset.survived

import pandas as pd

## Parameter Grid

As a model selection approach, the grid search allows you to investigate the optimal choice of model parameters by specifying the variations for each parameter.

For example, the SVC model has an error penalty parameter `C` in the model,
and we can specify the parameter grid as a dictionary, with the name of the parameter in question, then define the variations of the parameter in a list:

```python
param_grid = {'C': [1e3, 1e4] }
```

The dictionary allows us to supply the variations for different parameters:

```python
param_grid = {'C': [1e3, 1e4],
              'gamma': [1e-4, 1e-3], }
```

Conceptually, the parameter grid dictionary represents a cartesian product of the parameter variations, which contains all the configurations of the models that will be evaluated.  
In other words, the above `param_grid` helps to set up the following 4 models to evaluate:

```python
SVC(C=1e3, gamma=1e-4, ...)
SVC(C=1e3, gamma=1e-3, ...)
SVC(C=1e4, gamma=1e-4, ...)
SVC(C=1e4, gamma=1e-3, ...)
```

But sometimes this kind of cartesian product will generate too many models, more than desired.
We can also provide multiply dictionaries as alternative options.

Consider the difference between
```python
param_grid = [{'C': [1e3, 3e3],
              'gamma': [1e-4, 1e-3], },
             {'C': [5e2, 1e3],
              'gamma': [5e-5, 1e-4], }]
=> 2*2+2*2 = 8 models

```

and

```python
param_grid = [{'C': [1e3, 3e3, 5e2],
              'gamma': [1e-4, 1e-3, 5e-5], }]
=> 3*3 = 9 models
```

## Cross Validation

Not only does the Grid Search help you set up various configurations of models, it also sets up cross validation to provide more objective evaluation metrics of these models. `cv` parameter is used to specify number of cross validation folds used for evaluation.

## Practice

With the parameter grid and cross validation, the grid search results in a lot of computation.
However, we can leverage multiple processors available to accelerate the task, with `n_jobs` parameter, although be aware that in your Jupyter server learning environment, there may be resource limits to ensure fairness, i.e. you may not be able to use as many CPU cores as you can see, so we use a relatively low number in that parameter.

Now let's practice GridSearchCV usage, create a GridSearchCV, **named `clf`**:

1. Create at least 8 models with variations in `C`, `gamma` or your choice.
2. Use 5 fold cross validation.
3. Use 2 parallel jobs.

In [3]:
# Add your code for the above task here:   (Question #P001)
# ----------------------------------------

param_grid = [{'C': [1e3, 3e3],
              'gamma': [1e-4, 1e-3], },
             {'C': [5e2, 1e3],
              'gamma': [5e-5, 1e-4], }]
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid, cv=5, n_jobs=2)

Fit the model to loaded data `X` and `y`

In [4]:
# Add your code for the above task here:   (Question #P002)
# ----------------------------------------


clf.fit(X, y)



GridSearchCV(cv=5, estimator=SVC(class_weight='balanced'), n_jobs=2,
             param_grid=[{'C': [1000.0, 3000.0], 'gamma': [0.0001, 0.001]},
                         {'C': [500.0, 1000.0], 'gamma': [5e-05, 0.0001]}])

In [5]:
clf.best_estimator_

SVC(C=3000.0, class_weight='balanced', gamma=0.0001)

The GridSearchCV creates an abstract classifier, once trained with cross validation, the returned object itself can be used as a classifier that represent the optimal classifier within the given hyperparameter space.
For example, from the example above, `clf` will have many familiar methods like `.predict()`, `.score()`.

In addition, `clf.best_estimator_` gives you access to the best model chosen; `clf.best_score_` stores the accuracy score for the best model as well; `clf.cv_results_` provides details on cross validation.


In [6]:
clf.cv_results_

{'mean_fit_time': array([0.08720198, 0.17593098, 0.20091271, 0.47464433, 0.05518894,
        0.06364155, 0.07202444, 0.08655066]),
 'std_fit_time': array([0.02447042, 0.00562507, 0.01898928, 0.08204629, 0.01393411,
        0.00457791, 0.01138344, 0.02433957]),
 'mean_score_time': array([0.00618639, 0.00605459, 0.00603857, 0.00601635, 0.00665421,
        0.00631347, 0.00640664, 0.00613055]),
 'std_score_time': array([1.82422006e-04, 8.47071920e-05, 1.78202448e-04, 1.05679056e-04,
        1.60708087e-04, 2.25067977e-04, 1.70533568e-04, 1.72164405e-04]),
 'param_C': masked_array(data=[1000.0, 1000.0, 3000.0, 3000.0, 500.0, 500.0, 1000.0,
                    1000.0],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=[0.0001, 0.001, 0.0001, 0.001, 5e-05, 0.0001, 5e-05,
                    0.0001],
              mask=[False, False, False, False, False, False, False, False],
      

Now from `clf.cv_results_`, can you find where did the value of `clf.best_score_` come from?

Copy-paste the key/value pair from `clf.cv_results_` that shows the source of `clf.best_score_` below:

In [7]:
# Enter your answer below   (Question #P003)
# ----------------------------------------

print(clf.best_score_)

df = pd.DataFrame(clf.cv_results_)

best = df[df['rank_test_score'] == 1]

best.params
##{'C': 3000.0, 'gamma': 0.0001}



0.7786516853932584


2    {'C': 3000.0, 'gamma': 0.0001}
Name: params, dtype: object

Which was the fastest model to train? What were the parameters to the fastest model?

In [8]:
# Enter your answer below   (Question #P004)
# ----------------------------------------

df.mean_fit_time.min()


df
##{'C': 500.0, 'gamma': 5e-05}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.087202,0.024470,0.006186,0.000182,1000.0,0.0001,"{'C': 1000.0, 'gamma': 0.0001}",0.769663,0.730337,0.775281,0.797753,0.775281,0.769663,0.021903,2
1,0.175931,0.005625,0.006055,0.000085,1000.0,0.001,"{'C': 1000.0, 'gamma': 0.001}",0.764045,0.713483,0.764045,0.808989,0.769663,0.764045,0.030358,6
2,0.200913,0.018989,0.006039,0.000178,3000.0,0.0001,"{'C': 3000.0, 'gamma': 0.0001}",0.780899,0.735955,0.786517,0.808989,0.780899,0.778652,0.023729,1
3,0.474644,0.082046,0.006016,0.000106,3000.0,0.001,"{'C': 3000.0, 'gamma': 0.001}",0.758427,0.747191,0.747191,0.797753,0.780899,0.766292,0.019973,4
4,0.055189,0.013934,0.006654,0.000161,500.0,0.00005,"{'C': 500.0, 'gamma': 5e-05}",0.769663,0.719101,0.764045,0.792135,0.775281,0.764045,0.024359,5
5,0.063642,0.004578,0.006313,0.000225,500.0,0.0001,"{'C': 500.0, 'gamma': 0.0001}",0.769663,0.724719,0.764045,0.792135,0.764045,0.762921,0.021729,7
6,0.072024,0.011383,0.006407,0.000171,1000.0,0.00005,"{'C': 1000.0, 'gamma': 5e-05}",0.764045,0.713483,0.764045,0.792135,0.764045,0.759551,0.025474,8
7,0.086551,0.024340,0.006131,0.000172,1000.0,0.0001,"{'C': 1000.0, 'gamma': 0.0001}",0.769663,0.730337,0.775281,0.797753,0.775281,0.769663,0.021903,2


Run a prediction on the first 5 data samples from `X`.

In [9]:
# Enter your answer below   (Question #P005)
# ----------------------------------------


clf.predict(X[:5])




array([0, 0, 1, 0, 0])